In [1]:
import json
import logging
import re
import warnings
from datetime import datetime
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Generator, Literal, Type, TypeVar

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)


def create_path(path: str | Path) -> None:
    """
    Create parent directories for the given path if they don't exist.

    Parameters
    ----------
    path : str | Path
        The file path for which to create parent directories.

    """
    Path(path).parent.mkdir(parents=True, exist_ok=True)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/MyProjects/batch-process


In [5]:
from src.config import app_config

console.print(app_config)

AppConfig(
    data=Data(
        data_path='data/train.parquet',
        batch_data=BatchData(
            is_remote=True,
            remote_data_id='1lSKBgYQ6bgV4ARRpeRfPMpLgf1GgyWY8',
            download_path='data/downloaded_data.parquet',
            batch_mode=False,
            batch_size=100
        ),
        num_vars=['age', 'pclass', 'sibsp', 'parch', 'fare', 'survived'],
        cat_vars=['sex', 'embarked']
    ),
    db=DB(db_path='sqlite:///results.db', max_connections=5),
    model=Model(
        hyperparams=ModelHyperparams(n_splits=5, n_estimators=100, max_depth=10, random_state=42, test_size=0.2),
        artifacts=Artifacts(model_path='models/model.pkl')
    ),
    celery_config=CeleryConfig(
        task_config=TaskConfig(task_serializer='json', result_serializer='json', timezone='UTC', enable_utc=True),
        task_routes={
            'src.celery_pkg.tasks.email_tasks.*': QueueConfig(queue='email'),
            'src.celery_pkg.tasks.data_processing.*': QueueConfig(queue='data'),
            'src.celery_pkg.tasks.periodic_tasks.*': QueueConfig(queue='periodic'),
            'src.celery_pkg.tasks.ml_prediction_tasks.*': QueueConfig(queue='predict')
        },
        worker_config=WorkerConfig(
            worker_prefetch_multiplier=1,
            task_acks_late=True,
            worker_max_tasks_per_child=1000
        ),
        beat_config=BeatConfig(
            beat_schedule=BeatSchedule(
                cleanup_old_records=TaskAndSchedule(
                    task='src.celery_pkg.tasks.periodic_tasks.cleanup_old_records',
                    schedule=3600
                )
            ),
            health_check=TaskAndSchedule(task='src.celery_pkg.tasks.periodic_tasks.health_check', schedule=180)
        ),
        other_config=OtherConfig(
            result_expires=3600,
            task_compression='gzip',
            result_compression='gzip',
            result_backend_always_retry=True,
            result_persistent=True,
            result_backend_max_retries=3
        )
    ),
    api_config=APIConfig(
        title='Data Processing API',
        name='Data Processing API',
        description='API for processing large datasets',
        version='v0.2.5',
        status='healthy',
        batch_size=5,
        prefix='/api/v1',
        server=Server(host='0.0.0.0', port=8000, workers=1, reload=False),
        middleware=Middleware(
            cors=CORS(allow_origins=['*'], allow_credentials=True, allow_methods=['*'], allow_headers=['*'])
        )
    )
)

In [ ]:
import random
import string


# Define a function to generate a random id
def generate_random_id(length: int = 8) -> str:
    """
    Generate a random id string of a given length.

    Parameters
    ----------
    length : int, optional
        Length of the id string to generate. Defaults to 8.

    Returns
    -------
    str
        A random id string of the given length.
    """
    return "".join(random.choices(string.ascii_letters + string.digits, k=length))


# Define a function to generate a list of random person data
def generate_person_data(num_entries: int) -> list[dict[str, Any]]:
    """
    Generate a list of random person data.

    Parameters
    ----------
    num_entries : int
        Number of person data entries to generate.

    Returns
    -------
    person_data : list[dict]
        List of dictionaries, each containing person data.
    """
    sex_options: list[Literal["male", "female"]] = ["male", "female"]
    embarked_options: list[Literal["s", "c", "q"]] = ["s", "c", "q"]

    person_data = []

    for _ in range(num_entries):
        person = {
            "personId": generate_random_id(),
            "sex": random.choice(sex_options),
            "age": round(random.uniform(0.5, 80.0), 2),
            "pclass": random.randint(1, 3),
            "sibsp": random.randint(0, 5),
            "parch": random.randint(0, 5),
            "fare": round(random.uniform(5.0, 200.0), 2),
            "embarked": random.choice(embarked_options),
        }
        person_data.append(person)

    return person_data


# Generate a list of 10 random person data entries
person_data_list = generate_person_data(10)
print(person_data_list)

fp: str = "./data/sample_data.jsonl"

with open(fp, "w") as f:
    for person in person_data_list:
        f.write(json.dumps(person) + "\n")


In [ ]:
from sqlalchemy import delete, insert, select, update

from schemas import EmailSchema
from src.database.db_models import EmailLog, get_db_session, init_db

In [ ]:
init_db()

## [Docs](https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html)

### [Insert](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-bulk-insert-statements)

- Old API

```python
with get_db_session() as session:
    data_dict = input_data.model_dump()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}
```

<br>

- New API

```py
with get_db_session() as session:
    data_dict = input_data.model_dump()
    session.execute(insert(EmailLog), [data_dict])
```

In [ ]:
input_data: EmailSchema = EmailSchema(
    recipient="marketing@client.com",
    subject="Partnership Proposal",
    body="We would like to discuss a potential partnership opportunity.",
)
console.print(input_data)

In [ ]:
input_data.model_dump()

In [ ]:
with get_db_session() as session:
    data_dict = input_data.model_dump()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

In [ ]:
with get_db_session() as session:
    statement = session.query(EmailLog).where(EmailLog.created_at < datetime.now())
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

In [ ]:
input_data_2: EmailSchema = EmailSchema(
    recipient="emeka2@example.com",
    subject="test!!!",
    body="this is an example body",
    status="processing",
)
input_data_3: EmailSchema = EmailSchema(
    recipient="john.doe@example.com",
    subject="Meeting Reminder",
    body="Hi John, just a reminder about our meeting tomorrow at 10 AM.",
    status="processing",
)
input_data_4: EmailSchema = EmailSchema(
    recipient="info@company.org",
    subject="New Product Launch",
    body="Dear valued customer, check out our exciting new product!",
    status="sent",
    created_at=datetime(2025, 7, 10, 9, 0, 0),
    sent_at="2025-07-10T09:05:00",
)
console.print((input_data_2, input_data_3, input_data_4))

### [Bulk Insert](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-bulk-insert-statements)

- Old API

```py
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [_data.to_data_model_dict() for _data in (input_data_2, input_data_3, input_data_4)]
    session.bulk_insert_mappings(EmailLog, data_list)
```

<br>

- New API

```py
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [
        _data.to_data_model_dict()
        for _data in (input_data_2, input_data_3, input_data_4)
    ]
    session.execute(insert(EmailLog), data_list)
```

In [ ]:
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [_data.model_dump() for _data in (input_data_2, input_data_3, input_data_4)]
    session.execute(insert(EmailLog), data_list)

### Select

In [ ]:
# Select a single record
with get_db_session() as session:
    statement = select(EmailLog).where(EmailLog.id == 1, EmailLog.status == "pending")
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

In [ ]:
# Select all records
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

### [Update](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [ ]:
with get_db_session() as session:
    statement = (
        update(EmailLog)
        .where(EmailLog.id == 1)
        .values(status="sent", sent_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    )
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

### [Delete](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [ ]:
with get_db_session() as session:
    statement = delete(EmailLog).where(EmailLog.id == 2)
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

In [ ]:
from config import app_config

In [ ]:
beat_dict: dict[str, dict[str, Any]] = dict(app_config.celery_config.beat_config.beat_schedule.model_dump().items())

# Add the health_check
beat_dict["health_check"] = app_config.celery_config.beat_config.health_check.model_dump()


console.print(beat_dict)

In [ ]:
app_config.celery_config.beat_config.beat_schedule.model_dump().items()

In [ ]:
import json
from datetime import datetime
from typing import Any

from pydantic import BaseModel, field_serializer


class MyModel(BaseModel):
    name: str
    age: int
    role: str
    salary: float = 0.0
    others: Any | None = None

    @field_serializer("others")
    def serialize(self, value: Any) -> str:
        if isinstance(value, datetime):
            return value.isoformat()
        return json.dumps(value)


def my_func(name: str, **kwargs) -> MyModel:
    my_dict = {"name": name, **kwargs}
    return MyModel(**my_dict)


result = my_func(
    "Neidu",
    age=30,
    role="AI Engineer",
    friend="None",
    others=["Hi"],
    # others=datetime.now(),
)


In [ ]:
print(result.model_dump())

json.loads(result.model_dump()["others"])

In [ ]:
import json
import time
from datetime import datetime
from pathlib import Path
from typing import Any

import joblib

from celery import chord, current_task, group
from schemas import ModelOutput, MultiPersonsSchema, MultiPredOutput, PersonSchema
from src import PACKAGE_PATH, create_logger
from src.celery import celery_app
from src.database import get_db_session
from src.database.db_models import BaseTask, MLPredictionJob
from src.ml.utils import get_batch_prediction, get_prediction

logger = create_logger(name="ml_prediction")


@celery_app.task(bind=True, base=BaseTask)
def process_prediction_chunk(self, persons_data: list[dict[str, Any]], chunk_id: int) -> dict[str, Any]:  # noqa: ANN001
    """
    Process a chunk of ML predictions.

    Parameters
    ----------
    persons_data : list[dict[str, Any]]
        List of person data dictionaries for prediction
    chunk_id : int
        Unique identifier for this chunk

    Returns
    -------
    dict[str, Any]
        Dictionary containing chunk processing results and metadata
    """
    try:
        start_time = time.time()

        # Validate input data
        multi_persons = MultiPersonsSchema(persons=persons_data)  # type: ignore
        total_items = len(multi_persons.persons)

        # Load model once for the entire chunk
        model_dict_fp: Path = PACKAGE_PATH / "models/model.pkl"
        with open(model_dict_fp, "rb") as f:
            model_dict = joblib.load(f)

        # Process predictions
        prediction_results = []

        for i, person in enumerate(multi_persons.persons):
            # Update task progress
            current_task.update_state(
                state="PROGRESS",
                meta={"current": i + 1, "total": total_items, "chunk_id": chunk_id},
            )

            # Make individual prediction
            result: ModelOutput = get_prediction(person, model_dict)
            prediction_results.append(result.model_dump())

        processing_time = time.time() - start_time

        logger.info(f"Processed chunk {chunk_id} with {total_items} predictions in {processing_time:.2f}s")

        return {
            "chunk_id": chunk_id,
            "prediction_results": prediction_results,
            "processing_time": processing_time,
            "items_count": total_items,
            "status": "success",
        }

    except Exception as e:
        logger.error(f"Error processing prediction chunk {chunk_id}: {e}")
        raise self.retry(exc=e) from e


@celery_app.task
def combine_prediction_results(chunk_results: list[dict[str, Any]]) -> dict[str, Any]:
    """
    Combine results from multiple prediction chunks.

    Parameters
    ----------
    chunk_results : list[dict[str, Any]]
        List of chunk processing results

    Returns
    -------
    dict[str, Any]
        Dictionary containing combined prediction results
    """
    try:
        with get_db_session() as session:
            # Sort chunks by chunk_id
            sorted_results = sorted(chunk_results, key=lambda x: x["chunk_id"])

            # Combine all prediction results
            combined_predictions = []
            total_processing_time = 0
            total_items = 0

            for result in sorted_results:
                combined_predictions.extend(result["prediction_results"])
                total_processing_time += result["processing_time"]
                total_items += result["items_count"]

            avg_processing_time = round((total_processing_time / len(sorted_results)), 2)

            # Save to database
            job_data = {
                "job_name": "batch_ml_prediction",
                "input_data": json.dumps({"chunks": len(sorted_results), "total_items": total_items}),
                "output_data": json.dumps({"predictions": combined_predictions}),
                "processing_time": avg_processing_time,
                "prediction_count": total_items,
                "status": "completed",
                "completed_at": datetime.now(),
            }

            job = MLPredictionJob(**job_data)
            session.add(job)
            session.flush()

            logger.info(f"Combined {len(sorted_results)} chunks with {total_items} total predictions")

            return {
                "status": "completed",
                "total_chunks": len(sorted_results),
                "total_predictions": total_items,
                "avg_processing_time": avg_processing_time,
                "job_id": job.id,
                "predictions": combined_predictions,
            }

    except Exception as e:
        logger.error(f"Error combining prediction results: {e}")
        raise


@celery_app.task
def process_batch_predictions(persons_data: list[dict[str, Any]], chunk_size: int = 10) -> dict[str, Any]:
    """
    Process a large batch of ML predictions by splitting into chunks and using chord.

    Parameters
    ----------
    persons_data : list[dict[str, Any]]
        List of person data dictionaries for prediction
    chunk_size : int, optional
        Size of each processing chunk, by default 10

    Returns
    -------
    dict[str, Any]
        Dictionary containing batch processing dispatch information
    """
    try:
        # Split data into chunks
        chunks = [persons_data[i : i + chunk_size] for i in range(0, len(persons_data), chunk_size)]

        # Create a chord: process chunks in parallel, then combine results
        job = chord(
            group(process_prediction_chunk.s(chunk, i) for i, chunk in enumerate(chunks)),
            combine_prediction_results.s(),
        )

        result = job.apply_async()

        logger.info(f"Dispatched batch prediction job with {len(persons_data)} items in {len(chunks)} chunks")

        return {
            "status": "dispatched",
            "total_items": len(persons_data),
            "chunks": len(chunks),
            "chunk_size": chunk_size,
            "chord_id": result.id,
        }

    except Exception as e:
        logger.error(f"Error dispatching batch predictions: {e}")
        raise


@celery_app.task(bind=True, base=BaseTask)
def process_dlq_message(self, message_data: dict[str, Any]) -> dict[str, Any]:  # noqa: ANN001
    """
    Process a message from the dead letter queue.

    Parameters
    ----------
    message_data : dict[str, Any]
        Message data from DLQ

    Returns
    -------
    dict[str, Any]
        Dictionary containing DLQ processing results
    """
    try:
        # Validate the message data
        if "persons" in message_data:
            # Batch message
            record = MultiPersonsSchema(**message_data)
            message_type = "batch"
            item_count = len(record.persons)
        else:
            # Single message
            record = PersonSchema(**message_data)
            message_type = "single"
            item_count = 1

        # Log DLQ message to database (you might want to create a DLQ table)
        logger.warning(f"Processing DLQ message: {message_type} with {item_count} items")

        # For now, just log the DLQ data - you can extend this to save to a DLQ table
        with get_db_session() as session:
            job_data = {
                "job_name": f"dlq_{message_type}_processing",
                "input_data": json.dumps(message_data),
                "output_data": json.dumps({"status": "dlq_processed", "message_type": message_type}),
                "processing_time": 0.0,
                "prediction_count": 0,
                "status": "dlq_processed",
                "completed_at": datetime.now(),
            }

            job = MLPredictionJob(**job_data)
            session.add(job)
            session.flush()

            logger.info(f"DLQ message processed and logged with job_id: {job.id}")

            return {
                "status": "dlq_processed",
                "message_type": message_type,
                "item_count": item_count,
                "job_id": job.id,
            }

    except Exception as e:
        logger.error(f"Error processing DLQ message: {e}")
        raise self.retry(exc=e) from e
